In [74]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import os

In [75]:
# Getting the data 1
url = 'https://archive.ics.uci.edu/static/public/222/bank+marketing.zip'
dataset_path = '../datasets/'
dataset_file = 'bank+marketing.zip'
dataset_full_path = os.path.join(dataset_path, dataset_file)
response = requests.get(url)
with open(f'{dataset_path}bank+marketing.zip', 'wb') as file:
    for chunk in response.iter_content(chunk_size=1024):
        file.write(chunk)

with zipfile.ZipFile(f'{dataset_path}bank+marketing.zip') as zip_ref:
    zip_ref.extractall(f'{dataset_path}/bank+marketing')

with zipfile.ZipFile(f'{dataset_path}/bank+marketing/bank.zip') as zip_ref:
                zip_ref.extractall(f'{dataset_path}/bank+marketing')

In [80]:
bank = pd.read_csv(f'{dataset_path}/bank+marketing/bank-full.csv',sep=';')

In [81]:
df = bank[['age','job','marital','education','balance', 'housing','contact','day','month','duration', 'campaign','pdays', 'poutcome','y']]

In [86]:
# Question 1
# What is the most frequent observation (mode) for the column education?
# 
# unknown
# primary
# secondary
# tertiary
df['education'].value_counts()

education
secondary    23202
tertiary     13301
primary       6851
unknown       1857
Name: count, dtype: int64